In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from dateutil import parser
import pandas as pd
import time, re
import csv
import tweepy as tw
from csv import writer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Enter Twitter API Keys
access_token = "1387083235956506634-UIYJlKrPlrkmorsbXF7sB1Fbs0bYHw"
access_token_secret = "6vd1ZbwuVKkp5mG6GEsuZ8D5jGmWd71UU9bsdAwbK9o24"
consumer_key = "LPKQ4P9vWvXetyscRcgKeHNwj"
consumer_secret = "LQ3HfZeoycln1Dca5KHpyPYm2RdTiVMvVQVoFwTNyoiVqERDXq"

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

def urlRemover(txt):
    import re
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [3]:
def scrapTweet(query):
    driver = webdriver.Chrome(r'G:\My Drive\Covid19Data\chromedriver_win32_94\chromedriver')
    driver.base_url = query
    time.sleep(3)
    driver.get(driver.base_url)
    last_height=0
    height="window.scrollTo("+str(last_height)+","+str(last_height+2000)+");"
    tweet_ID_list=[]
    
    while(True):
        driver.execute_script(height)
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height=new_height
        height="window.scrollTo("+str(new_height)+","+str(new_height+2000)+");"
        #print(last_height)
        html_source = driver.page_source
        sourcedata= html_source.encode('utf-8')
        soup=bs(sourcedata)
        tweets=soup.body.findAll(["a"])
        for i in tweets:
            if '/status/' in str(i):
                clean_text= re.search('"auto" href=(.*) id=', str(i))
                if clean_text:
                    clean_text=clean_text.group(1).split('/')
                    tweet_ID_list.append(clean_text[-1][:-1])
                    print('row:',clean_text[-1][:-1])
    driver.close()
    return tweet_ID_list

In [4]:
def exactword(bigram):
    unibi=bigram.split(' ')
    result='https://twitter.com/search?q="'+unibi[0]+"%20"+unibi[1]+'"&src=typed_query&f=top'
    return result

def exactwordANyword(exactBigram,anywordList):
    #"construction zone" (wisconsin, OR WI, OR chicago, OR milwaukee)
    unibi=exactBigram.split(' ')
    anylist='%20('
    counter=0
    length_anywordList=len(anywordList)
    for i in (anywordList):       
            
        if counter<length_anywordList-1:
            anylist=anylist+i+"%2C%20OR%20"
           
        else:
            anylist=anylist+i+")"            
        counter+=1
    
    result='https://twitter.com/search?q="'+unibi[0]+"%20"+unibi[1]+'"'+anylist+'&src=typed_query&f=top'
    return result

In [6]:
df=pd.read_csv('bigram_search_for_tweet _usstate.csv',usecols=['Bigrams'])
dfCity=pd.read_csv('usstate.csv',usecols=['state','name'])
df.Bigrams.values[13:20]

array(['construction barrel', 'road closed', 'orange construction',
       'construction worker', 'road work', 'construction crew',
       'concrete barriers'], dtype=object)

In [ ]:
tweet_ID_list_all=[]
for i in df.Bigrams.values[20:]:    
    for j,k in zip(dfCity.state.values,dfCity.name.values):        
        query=exactwordANyword(i,[j,k])        
        scrapTweet(query)
        tweet_ID_list_all=tweet_ID_list_all+scrapTweet(query)
    
    

row: 1199321581782274048
row: 280802890765987840
row: 180378041325858817
row: 279563214742765568
row: 1199321581782274048
row: 280802890765987840
row: 180378041325858817
row: 279563214742765568
row: 1276499697159614466
row: 1276499697159614466
row: 1433103814765150211
row: 1433110092342120450
row: 1433103979437715459
row: 1433109582805553155
row: 1433103814765150211
row: 1433110092342120450
row: 1433103979437715459
row: 1433109582805553155
row: 122041082929360897
row: 655589170375618560
row: 122041082929360897
row: 655589170375618560
row: 1197489197717872641
row: 978929738583543808
row: 860594023853158400
row: 5779015873
row: 1422177091747868682
row: 1035165395056046080
row: 1383513732064571393
row: 1310733020500959232
row: 1422177091747868682
row: 1035165395056046080
row: 1383513732064571393
row: 1310733020500959232
row: 17325954406
row: 40796064214814720
row: 435846747739684864
row: 1384105812591665163
row: 583663405384540160
row: 1237686313329086465
row: 612937171142488064
row: 5760

row: 800020159511613440
row: 583436846400544768
row: 800734800361508864
row: 895745011920379905
row: 583438974003187712
row: 800712085323845634
row: 735079963624374272
row: 936346463591960578
row: 1035581949069455360
row: 936346480394358784
row: 1035583848896253952
row: 900806358039179265
row: 966783275908857856
row: 941442636359127046
row: 713351910095462400
row: 583978026053935105
row: 1226888281327112193
row: 583970947918606336
row: 1013568547124064256
row: 1301231380572057605
row: 800006896799404032
row: 811218228441739264
row: 94487759045144576
row: 811218228441739264
row: 94487759045144576
row: 979041044301058048
row: 72769055911714816
row: 1342172515175706625
row: 609333888834531328
row: 1008700197730439168
row: 86400608583692288
row: 523890435719057409
row: 547549862367989761
row: 677925181940043776
row: 800711221200568320
row: 747501483994677248
row: 811219565917208576
row: 800017446623031296
row: 29122882340
row: 978651486686244865
row: 1224324050077569024
row: 90080587473792

row: 1228067712334692352
row: 1232754354030931968
row: 1162893894272638977
row: 1002338992946229249
row: 1048317488499441664
row: 1155863284693778432
row: 989167598704189440
row: 1096922578357272576
row: 1157342768915779588
row: 1157513537029382144
row: 1445536503342383108
row: 1188130668426997760
row: 1435173498343739392
row: 1420354497055019008
row: 1420354860516405256
row: 1368570180062900227
row: 1264924341948555269
row: 1264946986488418309
row: 1313049756315406337
row: 1066918826762153984
row: 1125520617246081026
row: 1313831026460889089
row: 1313859734563422208
row: 1115722381287460867
row: 1024845697785520134
row: 968606691972481025
row: 1271419991494139904
row: 1413087445768237063
row: 1401297075292229637
row: 1401298035515301890
row: 1019351921369350144
row: 1019358126733082625
row: 1358790762528452608
row: 1358791018729119745
row: 1445535867209093135
row: 1416125654274351105
row: 1248691538307297286
row: 1418295826623782914
row: 1418781699027177476
row: 1161665643982151680
ro

row: 1201894398302326784
row: 335865752173293569
row: 383700528556822529
row: 614192784203870208
row: 1275236939428315137
row: 322185122692608000
row: 951166392664866816
row: 1040769880163680256
row: 498868442678972416
row: 676482671636279296
row: 740662694143918080
row: 1176687351344615430
row: 1387047193329111047
row: 907232303637782529
row: 859917984353202177
row: 398525231608123392
row: 240906205336305664
row: 489817909133582336
row: 382546516466237441
row: 198481651867717633
row: 382581629350576128
row: 540331830776315904
row: 313847798908977152
row: 1184808837347450880
row: 1265399056966340609
row: 935321780541100033
row: 623913573484511232
row: 1243270470180495366
row: 999094121049542656
row: 1177275493105590272
row: 661944206982668290
row: 917744778195988481
row: 1283871382397386754
row: 935321780541100033
row: 623913573484511232
row: 1243270470180495366
row: 999094121049542656
row: 1177275493105590272
row: 661944206982668290
row: 917744778195988481
row: 1283871382397386754
row

row: 7522438553
row: 6305420327
row: 509000376662900736
row: 13846924102
row: 317423540976644096
row: 786945066644275201
row: 460748528717803522
row: 712639338983522304
row: 655283413394530304
row: 507186810666356736
row: 635818419540107264
row: 1095003974619541504
row: 501855693612515328
row: 16324574647
row: 10179805617
row: 17285840296
row: 5951113921
row: 908177411522678784
row: 16324574647
row: 10179805617
row: 17285840296
row: 5951113921
row: 908177411522678784
row: 897377468209922049
row: 1313831026460889089
row: 1313859734563422208
row: 897377468209922049
row: 1313831026460889089
row: 1313859734563422208
row: 29006418538
row: 10868244216
row: 13350232463
row: 6931112494
row: 1413751205830987776
row: 834054040178872325
row: 1012762177017667584
row: 957007882876342272
row: 111418502560231424
row: 568421274923302912
row: 735483304191758336
row: 1388439702571786241
row: 1388528312683831300
row: 921732927221878785
row: 29006418538
row: 10868244216
row: 13350232463
row: 6931112494
ro

...

row: 1440684268821245963
row: 561936749845569537
row: 518188103291658241
row: 169555271176290304
row: 227546717678088193
row: 306508724288249856
row: 306515864906240000
row: 1372164243173281798
row: 1077852364235816960
row: 1167540974089900034
row: 1167543968227352582
row: 1167542243189755911
row: 1440065519323729927
row: 169555276779896832
row: 227546718969946112
row: 385949525602754560
row: 385949525602754560
row: 948622492725317632
row: 1083478702539882496
row: 215338923470237697
row: 997025245
row: 925899583976128512
row: 651750952689532928
row: 651772274027155456
row: 1269047029377925120
row: 385099266773221376
row: 948622492725317632
row: 1083478702539882496
row: 215338923470237697
row: 997025245
row: 925899583976128512
row: 651750952689532928
row: 651772274027155456
row: 1269047029377925120
row: 385099266773221376
row: 1269036788850413568
row: 1269048572248379393
row: 948622492725317632
row: 1083478702539882496
row: 215338923470237697
row: 997025245
row: 925899583976128512
row: 

row: 70325680390410240
row: 239803542330806273
row: 1276830553526976512
row: 1276979944082587649
row: 364811577024724992
row: 565230386922663937
row: 364249476187168769
row: 618542281629437952
row: 364249185253470208
row: 364254117402656768
row: 565248073157591042
row: 364249185253470208
row: 1405871040568328196
row: 565233020823937024
row: 498943218885853186
row: 498979033502449666
row: 1392482603576401924
row: 365922189754642433
row: 1292832307045433344
row: 1292832603452702720
row: 70325680390410240
row: 239803542330806273
row: 1276830553526976512
row: 498979033502449666
row: 1392482603576401924
row: 365922189754642433
row: 1292832307045433344
row: 1292832603452702720
row: 70325680390410240
row: 239803542330806273
row: 1276830553526976512
row: 1276979944082587649
row: 364811577024724992
row: 565230386922663937
row: 364249476187168769
row: 618542281629437952
row: 364249185253470208
row: 364254117402656768
row: 239803542330806273
row: 1276830553526976512
row: 1276979944082587649
row: 

row: 1433220224657072133
row: 781693814448599040
row: 251355053582057472
row: 1016734396240596995
row: 826807825569550336
row: 781603460110229504
row: 781645007195295744
row: 1433220224657072133
row: 781693814448599040
row: 251355053582057472
row: 1016734396240596995
row: 826807825569550336
row: 781603460110229504
row: 1433220224657072133
row: 781693814448599040
row: 251355053582057472
row: 1016734396240596995
row: 826807825569550336
row: 781603460110229504
row: 781645007195295744
row: 1038495473865187328
row: 886051398458023937
row: 1047950153468936193
row: 1038495473865187328
row: 886051398458023937
row: 1047950153468936193
row: 771856046432268288
row: 669943141873655809
row: 514436399815479298
row: 631572430046138368
row: 631965289819672577
row: 771856046432268288
row: 669943141873655809
row: 514436399815479298
row: 631572430046138368
row: 631965289819672577
row: 811286849868152836
row: 811298719404539904
row: 802658327859699712
row: 514436399815479298
row: 1123978922264285186
row: 

row: 1281415007381725184
row: 1281494619713347584
row: 1219639882487926787
row: 1340528172027183105
row: 1340522020665057280
row: 1340562859500048384
row: 1340572575882199041
row: 1340615481418399745
row: 1340504026144677888
row: 1340503714650488832
row: 1340564766754889728
row: 385903089141551104
row: 383378934730981376
row: 383543719909400576
row: 8745230802
row: 203639362620096513
row: 149536116373065728
row: 870566098223091712
row: 847087960797384704
row: 1305373471438811137
row: 1305375834857811973
row: 383580247238909952
row: 1029015632409513984
row: 1029021234850394112
row: 676757007383330816
row: 1413411863421808642
row: 385903089141551104
row: 383378934730981376
row: 383543719909400576
row: 8745230802
row: 203639362620096513
row: 149536116373065728
row: 870566098223091712
row: 847087960797384704
row: 1305373471438811137
row: 1305375834857811973
row: 383580247238909952
row: 1029015632409513984
row: 1029021234850394112
row: 676757007383330816
row: 1413411863421808642
row: 115693

row: 359143368279851010
row: 359148187015057408
row: 1380986903327956995
row: 757517727154200576
row: 867683434579230720
row: 1303946978955603968
row: 764791160409915393
row: 1177122419091431424
row: 1215193055260823553
row: 765120274907922432
row: 764850532729126912
row: 1177122406063890432
row: 1215193081219252224
row: 8124253974
row: 5301321251
row: 867683434591858688
row: 1137727568461746177
row: 840084352935514113
row: 388219609796452352
row: 395845796370731008
row: 780377424533913600
row: 757517727171051520
row: 1130176177493172225
row: 1303946923079094272
row: 231456129765490688
row: 474717936330866691
row: 1306317330578694145
row: 1207414259111456768
row: 1062156294205923328
row: 255365807956168706
row: 764787379991416832
row: 357538296253001729
row: 765120274916319232
row: 757517727154274304
row: 1303946887561728000
row: 867683434713448451
row: 384958391384158208
row: 764791610861314048
row: 765120274878623745
row: 1177122431267483649
row: 753519100744364032
row: 1153741194381

In [11]:
pd.DataFrame({'tweetID':[("id"+i) for i in tweet_ID_list_all]}).to_csv('tweetID_usstate.csv',index=None,mode='a')

In [12]:
def tweetfromAPi(idlist): 
    c=0
    lengthobsub=100
    while lengthobsub>99:
        #lengthobsub=len(tweet_ID_list_all[c:c+100])
        tweetList=idlist[c:c+100]
        #print(tweetList)
        lengthobsub=len(tweetList)
        c+=100
        try:
            tweets = api.statuses_lookup(tweetList,tweet_mode="extended")
            #print(tweets)
            for tweet in tweets:
                #print(tweet.created_at.date())
                if tweet.place:
                    place=["id"+tweet.id_str,tweet.created_at.date(),tweet.place.full_name+","+tweet.place.country,tweet.full_text]       
                    pd.DataFrame(data=[place ]).to_csv('WZ.csv',mode='a',header=None,index=None)
                else:
                    place=["id"+tweet.id_str,tweet.created_at.date(),"None",tweet.full_text]
                    #print(place)
                    pd.DataFrame(data=[place ]).to_csv('WZ.csv',mode='a',header=None,index=None)

        except:      

            pass


In [13]:
tweet_id_df=pd.read_csv('tweetID_usstate.csv')
tweetID=[i[2:] for i in tweet_id_df.tweetID.values]
tweetfromAPi(tweetID[19747:])
#tweetfromAPi([1143838354078703616])

In [ ]:
# suffle the downloaded tweets
df=pd.read_csv('WZ.csv')
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('tweet_usstate.csv', index=None)